In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("../data/cars_dataset.csv")

data.head(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [3]:
## checking for na values
data.isna().sum().sum()

0

In [5]:
# grabbing median year
median_year = data['model'].median()
median_year

76.0

In [16]:
# setting up the filtered data to work with
work_set = data[(data['model'] >= median_year) & ((data['origin'] == 1) | (data['origin'] == 3))]
jpn_cars = work_set[work_set['origin'] == 3]["acceleration"]
usa_cars = work_set[work_set['origin'] == 1]["acceleration"]

# declare significance level

a = 0.05
#comparing shapes
print(data.shape)
print(work_set.shape)
print("Japanese cars count : ", jpn_cars.shape)
print("American cars count : ", usa_cars.shape)
work_set.head(5)


(398, 9)
(179, 9)
Japanese cars count :  (54,)
American cars count :  (125,)


,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car_name
184,25.0,4,140.0,92.00,2572.0,14.9,76,1,capri ii
185,26.0,4,98.0,79.00,2255.0,17.7,76,1,dodge colt
187,17.5,8,305.0,140.0,4215.0,13.0,76,1,chevrolet chevelle malibu classic
188,16.0,8,318.0,150.0,4190.0,13.0,76,1,dodge coronet brougham
189,15.5,8,304.0,120.0,3962.0,13.9,76,1,amc matador


In [17]:
#based upon the same data in the test
#we wanna do the independent t-test
t_test, p_value = stats.ttest_ind(usa_cars, jpn_cars, alternative="greater")

print(t_test, p_value)

# appears that the null is more likely

-1.0163338896871557 0.8445711798616595
